# **Technical Prerequisites**

This document is a Jupyter Notebook. A Jupyter Notebook is a document that contains a mix of static text (like the one you are reading) and executable code (like the one you see if you scroll down). 

The static text can be enriched with mathematical formulas and media such as images or videos, making notebooks a powerful tool to write and present code with explanations. \
The code is usually (and in our case, it is) Python, but you might be interested in knowing that it can be R and a wealth of other programming languages.

The figure below illustrates the basic anatomy of a Jupyter Notebook as I see it on my laptop:

<p align="center">
<img src="./files/jupyter-notebook-anatomy.png" width="1100"/>
</p>

**Text cells:**
- To write text in a text cell, double-click on it or press `Enter` after a single click. This will put the text cell into edit mode
- Once you are done editing, press `Ctrl + Enter` to render your text
- You can switch between edit and rendered mode as many times as you like

**Code cells:**
- To write code in a code cell, just click on it and start typing
- To run a code cell, click on it to select it, then press `Ctrl + Enter`

---

Feel free to delete this cell once you are done learning its content. To do so, you can select it and press `D` twice rapidly. \
If you regret deleting it, you can alway press `Ctrl + Z` to restore it (but it only works if you do it promptly).

# **Introduction**

Students write text here, explaining:
 - What it means to preprocess data and what is a preprocessing pipeline
 - What are the main artefacts that can be found in the TMS-EEG signal and their causes
 - **Optional**: a personal comment or opinion on the problems posed by preprocessing, based on what you heard in class during both theoretical lessons and the first _briefing_

In [1]:
from pathlib import Path                            # to build a bridge between Python and the filesystem 

import numpy as np                                  # to perform array operations
import matplotlib                                   # this and the following to draw plots
import matplotlib.pyplot as plt
matplotlib.use('Qt5Agg')                            # to make plots interactive

import mne                                          # to read and manipulate EEG data

from scripts import utils                           # custom functions to shorten the code in this notebook

# **1. Basic Preprocessing (Assignment 1, Deadline 26/11/2025)**

Steps of the Basic Prepocessing:
1. Loading data;
2. Inspecting data (to understand their shape and content):
  - shape: number of channels and time length;
  - acquisition parameters: sampling rate (frequence in which computer read/save signal data);
  - appearence: are the raw data visibly dirty and how much?
3. Adjusting data (if present, drop-out non-EEG channels, e.g., EMG, EOC, ECG, etc.);
4. Interpolating the TMS pulse artefact;
5. High-pass filtering.

## **1.1. Loading and Inspecting Data**

**Optional but welcome:** students write here any considerations about this step.

Comments: 
1. **You should know the format of your data** (we have BrainVision format, very popular for the EEG data)
BrainVision format represents the EEG data in 3 type of files:
   1) _.eeg file_: EEG time series;
   2) _.vhdr_: header file = metadata file;
   3) _.vmrk_: marker file = temporal information about the event;
2. The function that reads the raw data in BrainVision format is `read_raw_brainvision()` that we request from the `io` module of the `MNE package`.

In [3]:
data_dir = Path("data")
subject_and_condition = "S02C1_M1"

file_of_interest = list(data_dir.glob(pattern=f"{subject_and_condition}.vhdr"))
print(file_of_interest, "|", type(file_of_interest), "|", file_of_interest[0])

[WindowsPath('data/S02C1_M1.vhdr')] | <class 'list'> | data\S02C1_M1.vhdr


In [4]:
eeg_data = mne.io.read_raw_brainvision(vhdr_fname=file_of_interest[0])

# take read_raw_brainvision f-n from the io module of mne package 

Extracting parameters from data\S02C1_M1.vhdr...
Setting channel info structure...


In [23]:
eeg_data

# Why 72 channels: bc you need to have reference electrodes
# they took the avarage of channels

<RawBrainVision | S02C1_M1.eeg, 72 x 3307600 (661.5 s), ~63 KiB, data not loaded>

In [ ]:
eeg_data.ch_names

## **1.2. Drop Unwanted Channels**

**Compulsory:** students write here the functional significance of this step (that is, why we do it).

_Why we are clearing out/exclude some Channels?_ \
In dataset we might have not only onfo from EEG channels, but also from additional channels. In our case there were EOG and FDI channels - non-EEG channels.

We can do it manually, which is lame, so we can do it faster with code.

In [5]:
channels_to_drop = []

for channel_name in eeg_data.ch_names:
    try:
        int(channel_name[-1])
    except ValueError:
        if channel_name[-1] != "z": 
            channels_to_drop.append(channel_name)
print(f"Channels to drop: {channels_to_drop}")

eeg_data = eeg_data.drop_channels(ch_names=channels_to_drop)

Channels to drop: ['EOG', 'FDI']


In [9]:
eeg_data

<RawBrainVision | S02C1_M1.eeg, 70 x 3307600 (661.5 s), ~61 KiB, data not loaded>

## **1.3. Set Channels Location**

**Compulsory:** students write here the functional significance of this step (that is, why we do it).

_Why do we need this step?_ \
Beacause without it we do not have information about the shape of the head and about the position of the channels on it.

_Why do we need this info?_ \
Because without it we cannot do following:
1. Source reconstruction and other analyses;
2. Visualization (e.g., topoplots or scal maps).



In [6]:
easycap_m1_montage = mne.channels.make_standard_montage(kind="easycap-M1")
easycap_m1_montage.plot()
eeg_data.set_montage(montage=easycap_m1_montage,
                     on_missing="ignore")   # apply the montage

<RawBrainVision | S02C1_M1.eeg, 70 x 3307600 (661.5 s), ~86 KiB, data not loaded>

In [ ]:
easycap_m1_montage.get_positions()

**Compulsory:** here, students describe and comment the plot generated by the code below. In particular, I would like to read:

- What each row represents: 
   each row represents a channel that "represents the electrical potential at that point on the scalp relative to a reference electrode or a common reference point" 

- What are the $x$ and $y$ of each row ("where the x-axis denotes time and the y-axis represents the magnitude"), and their units of measurement (x=time: seconds; y= voltage magnitude: mV, microvolts) _(what is the difference betweem magnitude, amplitude and power?)_

- A description of the artefacts you see, and their likely causes:
    - peaky little ones, on the channels near to face: blinking;
    - super high magnitude, super high frequence bundles: muscle artefacts;
    - too regular |||||||||||||| pattern all over the channel: the channel is all noise, possibly too much environmental noise;
    - huge | like part that melt with other channels in one big line right at the time of the TMS pulse onset - TMS pulse artefact.

In [6]:
eeg_data.plot()

Using qt as 2D backend.


**Compulsory:** here, students define event markers and explain why they are important for EEG data analysis

Event markers / triggers - the time stamps that locate the events of the experimental interest on an EEG trace (e.g., TMS pulse). We need it to interpolate the TMS pulse artefact and to be able to analyse evoked brain activity.

The data about the event markers is contained in the `.vmrk` file, where these details are stored:
 - an ordinal number (e.g., 200)
 - a type of the stimulus
 - a name
 - a time of occurance (in unit of data points)
 - a duration (in unit of data points)
 - info about which channels were involved

E.g.: Mk200 = stimulus, S 54, 31.8937630, etc.


**Optional:** any comments that go beyond a mere definition are welcome. For example, this could be comments about the structure of event arrays and what it tells us about the nature of events

In [7]:
events_from_annotations, events_dict = mne.events_from_annotations(raw=eeg_data)
events_from_annotations = events_from_annotations[2:]

Used Annotations descriptions: ['New Segment/', 'Stimulus/S 54', 'Stimulus/S255']


## **1.4. Interpolate the Pulse Artefact**

**Compulsory:** students write here the functional significance of this step (that is, why we do it).

Why we do this:
TMS pulse is large and seriously interferes with EEG recording system. This pulse flat out all other signal in our visualization plot and mess-up all the brain data in this time period, so we have to get rid of this part of the signal. As we cut out this TMS-induced ERP, we have a hole in the data => we have to interpolate this part (i.e., to estimate the function (value) in the given interval, given values from the preceeding and following intervals).



In [8]:
pre_interpolation_epochs = mne.Epochs(raw=eeg_data,
                                      events=events_from_annotations,
                                      tmin=-1.1,
                                      tmax=0.5,
                                      baseline=None)
pre_interpolation_epochs_tep = pre_interpolation_epochs.average()
pre_interpolation_epochs_tep.plot(); # this is our pulse artifact that flat all of the other channels

Not setting metadata
200 matching events found
No baseline correction applied
0 projection items activated


In [9]:
post_interpolation_eeg = mne.io.read_raw(fname="data/post_2_5_interpolation_eeg.fif",
                                         preload=True)
post_interpolation_epochs = mne.Epochs(raw=post_interpolation_eeg,
                                       events=events_from_annotations,
                                       tmin=-1.1,
                                       tmax=0.5,
                                       baseline=None)
post_interpolation_tep = post_interpolation_epochs.average()
post_interpolation_tep.plot();

Opening raw data file data/post_2_5_interpolation_eeg.fif...
    Range : 0 ... 3307599 =      0.000 ...   661.520 secs
Ready.
Reading 0 ... 3307599  =      0.000 ...   661.520 secs...
Not setting metadata
200 matching events found
No baseline correction applied
0 projection items activated


## **1.5. High-pass filtering**

**Compulsory:** students write here the functional significance of this step (that is, why we do it) and potential caveats (that is, one-two things that can happen if you do this step unproperly).

High-pass meaning "cut off everything that is lower and leave everything that is higher". Usually, you do band-pass filtering (more than some Hz and less than other Hz) + notch filtering (you cut of a specific frequency, e.g., in the EU 50Hz to get rid of unwanted electric inference)

Why we do it: 
 - to cut off noise frequencies (artefacts) that EEG picked up during the recording, e.g.: sweat artefacts => to keep SNR as good as possible.
  
Potential caveats:
 - you might lose important data if your High-pass filtering is too high, as we have some slow brain frequencies;
 - no filtering or preprocessing in general can save very bad quality data: "garbage in - garbage out" principle.

In [10]:
post_filtering_eeg = mne.filter.filter_data(data=post_interpolation_eeg.get_data(),
                                            sfreq=post_interpolation_eeg.info["sfreq"],
                                            l_freq=0.1,
                                            h_freq=None,
                                            method="iir",
                                            iir_params=None,
                                            copy=True,
                                            phase="zero")
post_filtering_eeg = mne.io.RawArray(data=post_filtering_eeg,
                                     info=post_interpolation_eeg.info)
post_filtering_epochs = mne.Epochs(raw=post_filtering_eeg,
                                   events=events_from_annotations,
                                   tmin=-1.1,
                                   tmax=0.5,
                                   baseline=None)
post_filtering_tep = post_filtering_epochs.average()
post_filtering_tep.plot();

Setting up high-pass filter at 0.1 Hz

IIR filter parameters
---------------------
Butterworth highpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 8 (effective, after forward-backward)
- Cutoff at 0.10 Hz: -6.02 dB

Creating RawArray with float64 data, n_channels=70, n_times=3307600
    Range : 0 ... 3307599 =      0.000 ...   661.520 secs
Ready.
Not setting metadata
200 matching events found
No baseline correction applied
0 projection items activated


# **2. Independent Component Analysis (ICA) (Assignment 2, Deadline 05/12/2025)**

## **2.1. Rationale**

**Compulsory:** students write here the functional significance of this step (that is, why we do it) and expand on its rationale, devoting particular attention to discussing the independence assumption and why its application to brain data is debatable.

\
ICA is a mathematical procedure to decompose a signal into a set of factors that respect the following constraints:
 - they are linearly independend of one another - that is, no factor can be written as a linear function of the other;
 - the original signal can be reconstructed as a combination od said factors.

 **Goal of ICA in EEG**: \
 isolating brain from non-brain signals, in particular, eye and muscle artefacts 


 **Typical EEG ICA steps**:
 1. Fit an ICA model to the data, i.e. run code that factors the signal into independent components.
 2. Visualize the resulting components with topoplots and voltage time series.
 3. Evaluate the plots to seek sign of non-brain activity.
 4. Factor out (exclude) non-brain components.
 5. Reconstruct the signal using only brain components.

\
ICA assumes that signal is a combination of independent sourses, but _this is a very questionable concept_, because, for example:
- brain areas interact with each other;
- heartbeat modulates brain activity and vice versa;
- muscle contruction might come from the brain.

Besides, while ICA can factor brain signal into statistically independent components, those components do not necesserily make neuroscientific sense + "Not all components are meaningful. More specifically: most components are not meaningful."

For this, **eyeblinks might be considered as the most independent component**, even if they also originate from the brain as they has three characteristics that make it reasonable to factor them out:
1. Lack of connection with cognitive dynamics.
2. Reative amplitude of the eyeblinks signal is much larger than the brain one.
3. Lack of better methods.



## **2.2. Fitting**

In [11]:
# ica = mne.preprocessing.ICA(random_state=0)
# ica.fit(post_filtering_epochs)

ica = mne.preprocessing.read_ica(fname="data/fitted_ica.fif")

Reading c:\Users\korsu\OneDrive\Desktop\brainstim-multimodal-main\data\fitted_ica.fif ...
Now restoring ICA solution ...
Ready.


## **2.3. Components Selection**

**Compulsory:** students select components whose most likely source are eye movements and justify their choices here, including a picture of the selected component(s).

To help you tell components apart, you can refer to [this](https://labeling.ucsd.edu/tutorial/labels) tutorial by the developers of ICLabel: an algorithm to automatically classify independent components that came out of the Swartz Center for Computational Neuroscience, University of California San Diego ([Pion-Tonachini et al., 2019](https://www.sciencedirect.com/science/article/pii/S1053811919304185)).

To include a picture of the selected component(s), just take a screenshot, save it somewhere and change the path below (`files/sample-ic.png`) with the path to your screenshot. 


\
**EYEBLINK ARTEFACTS**

`IC000`: 
 - PSD: power concentration at low frequencies
 - scalp topography: near eyes
 - time series: evident vertical blinks artefacts

`IC021`:
 - PSD: kinda power concentration at low frequencies, but contaminated with other artefacts
 - scalp topography: near eyes (right side)
 - time series: evident vertical blinks artefacts at epoches 118 and 123

I also suspected `IC008` (the position in the scalp topography and it seems like there is a similar to other two pattern for low frequencies power distribution in PSD, but idk), but I couldn't find anything particular in the time series.

<p align="center">
<img src="./files/eyeblink1.png" width="1000"/>
</p>

<p align="center">
<img src="./files/eyeblink2.png" width="1000"/>
</p>


Most of other channels are artefacts as well, for example, it feels like everything starting from 20th-s ICs are definetely artefacts.

In [23]:
ica.plot_components(inst=post_filtering_epochs, picks=range(25))

Using data from preloaded Raw for 200 events and 8001 original time points ...
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
200 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 200 events and 8001 original time points ...
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
200 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 200 events and 8001 original time points ...
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
200 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 200 events and 8001 original time points ...
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
200 matching events found
No baseline correction applied
0 projection items activated
Using data f

<MNEFigure size 975x1050 with 25 Axes>

In [31]:
ica.plot_sources(inst=post_filtering_epochs)

Not setting metadata
200 matching events found
No baseline correction applied
0 projection items activated


In [29]:
components_to_reject = utils.select_items(item_type="ica")  # to reject the components I do not need

In [26]:
components_to_reject

[0, 21]

In [30]:
ica.exclude = components_to_reject
ica.apply(post_filtering_epochs.load_data())
post_ica_epochs = post_filtering_epochs
post_ica_tep = post_ica_epochs.average()
post_ica_tep.plot();

Applying ICA to Epochs instance
    Transforming to ICA space (64 components)
    Zeroing out 2 ICA components
    Projecting back using 64 PCA components


# **3. Manual Artifact Rejection (Assignment 3, Deadline 12/12/2025)**

## **3.1. Rationale**

**Compulsory:** students write here the functional significance of this step (that is, why we do it) and expand on the criteria to follow for its execution. 

## **3.2. Execution**

In [ ]:
post_ica_epochs.plot()

# **4. Computing & Assessing a TEP (Assignment 4, Deadline 19/12/2025)**

## **4.1. Rationale**

**Compulsory:** students write here the functional significance of this step (that is, why we do it) and expand on the criteria to follow for its execution. 

## **4.2. Execution**

**Optional:** Student can write here any comment or consideration that goes beyond the minimum required, for example:
- "I think filtering is `{good/bad}` because..."
- "I think these data were particularly `{dirty/clean}` because..."
- Anything else you might feel like saying about the preprocessing you have done

Do not be afraid to make such comments: there is no right or wrong and they do not count for the vote. At most, a great comment could make you a _cum laude_ candidate, but I really just want to see if you have a personal opinion on the subject.